In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import fbeta_score, make_scorer
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

/home/aanax/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
location = ''
events = pd.DataFrame.from_csv(location+"user_activity.csv",index_col=None)
structure = pd.DataFrame.from_csv(location+"structure.csv",index_col=None)
targets = pd.DataFrame.from_csv(location+"targets.csv",index_col=None)
events_test = pd.DataFrame.from_csv(location+"user_activity_test.csv",index_col=None)

In [4]:
targets.head()

,user_id,score,passed,distinction
0,1,0,0,0
1,2,82,0,0
2,3,89,0,0
3,4,9,0,0
4,6,26,0,0


In [5]:

def encodeact(acts):
    new=np.zeros((4))
    for i in acts:
        if i[0]=='d':
            new[0]=1
        if i[0]=='v':
            new[1]=1
        if i[0]=='s':
            new[2]=1
        if i[0]=='p':
            new[3]=1
        #print(new)
    #while len(new)<4:
    #    new.append(0)
    #new.sort()
    return new

In [6]:
#step to step average time.
#step to step average time derivative?

encoded = pd.DataFrame([i for i in events["action"].apply(encodeact)],columns = ["discover","view","start","pass"])
events = pd.concat([events,encoded],axis=1)

encoded_test = pd.DataFrame([i for i in events_test["action"].apply(encodeact)],columns = ["discover","view","start","pass"])
events_test = pd.concat([events_test,encoded_test],axis=1)

In [7]:
from scipy.interpolate import InterpolatedUnivariateSpline

In [58]:
def getX(ev):
    'Функция формирует входные вектора для алгоритма RandomForest'
    sc  = ev.groupby('user_id')
    mintime = np.array(sc.time.max()).reshape((-1,1))
    maxtime = np.array(sc.time.min()).reshape((-1,1))
    
    
    numsess=[]#pd.DataFrame(columns=["numsess"])
    avgsess=[]#pd.DataFrame(columns=["avgsess"])
    avgtime=[]#pd.DataFrame(columns=["avgtime"])
    derivend=[]
    derivavg=[]
    maxdelay=[]
    lasttime=[]
    lasttime1=[]
    iscoder=[]
    isnumber=[]
    isstring=[]
    for name,group in sc:
       #print(group["time"][0:5])
        base = np.array(group["time"]).reshape(-1,1)
       #print("+++++++++++")
       #print(base)
        shifted = [0]+list(group["time"])
        shifted.pop()
        shifted = np.array(shifted).reshape(-1,1)
        ststime = base - shifted
        if len(ststime)>=3:
            ststime[0] = np.mean(ststime[1:])
            avgststime = np.mean(ststime)

           #print("DDDDDDDDDDDDDD")
           #print(avgststime)

            sesstimes = [i for i in ststime if i>=avgststime]
            avgsesstime = np.mean(sesstimes)
            sessions = len(sesstimes)

            #f = InterpolatedUnivariateSpline(x=list(range(0,len(sesstimes))), y = sesstimes, k=2)
            #f = np.poly1d(in)
            #dfdx = f.derivative()
            sesstimes = np.array(sesstimes).reshape(-1,1)
            #print(sesstimes.shape)
            
            #lasttime.append(sesstimes[-1])
            #lasttime1.append(ststime[-1])
            
            if sesstimes.shape[0] >=2:
                
                dfdx = np.gradient(sesstimes,axis=0)
            else:
                dfdx = [0]
            
        
        else:
            avgststime = 1000000
            sessions=1
            avgsesstime=1000000
            dfdx=[0]
            
        numsess.append(sessions)
        avgsess.append(avgsesstime)
        avgtime.append(avgststime)
        derivend.append(dfdx[-1])
        maxdelay.append(max(ststime))
        lasttime.append(sesstimes[-1])
        lasttime1.append(ststime[-1])
        #
        passed = group[group["action"] == 'passed']
        iscoder.append(np.array(passed["step_type"] == 'code').sum())
        isnumber.append(np.array(passed["step_type"] == 'number').sum())
        isstring.append(np.array(passed["step_type"] == 'string').sum())
        #derivend.append(dfdx[len(dfdx)-2])
        #derivend.append(dfdx[len(dfdx)-3])
        #derivend.append(dfdx[len(dfdx)-4])
        derivavg.append(np.mean(dfdx))
        #for i in range(0,len(ststime)):
            #if ststime[i]<avgststime
        
    numsess=np.array(numsess).reshape((-1,1))
    avgsess=np.array(avgsess).reshape((-1,1))
    avgtime=np.array(avgtime).reshape((-1,1))
    derivavg=np.array(derivavg).reshape((-1,1))
    derivend=np.array(derivend).reshape((-1,1))
    maxdelay=np.array(maxdelay).reshape((-1,1))
    lasttime=np.array(lasttime).reshape((-1,1))
    lasttime1=np.array(lasttime1).reshape((-1,1))
    iscoder=np.array(iscoder).reshape((-1,1))
    isnumber=np.array(isnumber).reshape((-1,1))
    isstring=np.array(isstring).reshape((-1,1))
    print("1deal",avgtime.shape)
    print("1",lasttime.shape)
    print("2",lasttime1.shape)
    return (sc.sum().index, np.hstack([sc.sum()[["step_cost" ,"view","start","pass"]].values,
                                       mintime,#good
                                       maxtime,
                                       maxtime-mintime,#good
                                       numsess, #gd?
                                       #avgsess,#bad
                                       avgtime,#good
                                       derivend,
                                       derivavg,
                                       maxdelay,
                                       lasttime,
                                       #lasttime1,#bad
                                       #isnumber,#bad
                                       isstring,
                                       iscoder
                                      ] )) #   "discover",
    #return (sc.sum().index, np.hstack([iscoder,mintime,maxtime,sc.sum()[["start","pass"]]]))


In [59]:
ind, X = getX(events)

1deal (16625, 1)
1 (16625, 1)
2 (16625, 1)


In [60]:
ind_test, X_test = getX(events_test)

1deal (4255, 1)
1 (4255, 1)
2 (4255, 1)


In [61]:
Y = targets.passed.values

In [62]:
Xtr,Xval,Ytr,Yval = train_test_split(X,Y,test_size=0.1,random_state=128)

In [63]:
#parameters = {'max_depth':list(range(4,10)), 'learning_rate':list(np.linspace(0.0001,0.01,num=4)),'n_estimators':list(range(10,10000,310))}
#boostmodel = xgb.XGBClassifier()
#clf = GridSearchCV(boostmodel, parameters,scoring = "f1",n_jobs=3)

In [64]:
#clf.best_params_

In [65]:
#with warnings.catch_warnings():
#    warnings.simplefilter("ignore")
    # Do stuff here
#    clf.fit(Xtr,Ytr)

In [66]:
boostmodel1 = xgb.XGBClassifier(max_depth=6,learning_rate=0.0001,n_estimators=5900,base_score=0.6)#,reg_alpha=0.1)
boostmodel1 = boostmodel1.fit(Xtr, Ytr)

In [67]:
ttst=boostmodel1.booster()

In [68]:
#boostmodel05 = xgb.XGBClassifier(max_depth=6,learning_rate=0.0001,n_estimators=5900,base_score=0.5)#,reg_alpha=0.1)
#boostmodel05 = boostmodel05.fit(Xtr, Ytr)

In [69]:
#print(f1_score(Yval, boostmodel05.predict(Xval)))

In [70]:
#Оценим метрику на validation
print(f1_score(Yval, boostmodel1.predict(Xval)))

#without bad features does better on val but worse on lb

0.411764705882


In [447]:
features = ["step_cost","discover","view","start","pass",
                                       "mintime",
                                       "maxtime",
                                       "maxtime-mintime",
                                       "numsess",
                                       "avgsess",
                                       "avgtime",
                                       "derivend",
                                       "derivavg",
                                       "maxdelay",
                                       "lasttime",
                                       "lasttime1"
                                       ,"isnumber","isstring","iscoder"]

In [417]:
def create_feature_map(fmap_filename, features):
    '''
    features: enumerable of feature names
    '''
    outfile = open(fmap_filename, 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

create_feature_map('xgb.fmap', features) 

In [78]:
from matplotlib import pyplot
import seaborn as sns

In [68]:
a=ttst.get_score(fmap = 'xgb.fmap', importance_type = 'cover')#[list(ttst.get_score(fmap = 'xgb.fmap', importance_type = 'cover').keys())]

In [83]:
import pandas as pd

In [129]:

#clf.best_estimator_

#boostmodel1.feature_importances_
a=pd.DataFrame(ttst.get_score(fmap = 'xgb.fmap', importance_type = 'gain'),np.arange(1))
#sns.plt.([i[0] for i in a.items()],[i[1] for i in a.items()])
#pyplot.bar(range(len(boostmodel1.feature_importances_)), boostmodel1.feature_importances_)
a.plot(kind="bar",colormap="Pastel1")
pyplot.show()

In [91]:
pd.DataFrame(a,np.arange(1))

,avgsess,avgtime,derivavg,derivend,discover,iscoder,isnumber,lasttime,lasttime1,maxdelay,maxtime,maxtime-mintime,mintime,numsess,pass,start,step_cost,view
0,412.244569,72.459457,66.416672,105.634197,56.332259,1739.357736,39.460066,32.961805,42.192599,129.180926,236.760132,140.44222,474.030209,22.003198,30.683342,296.324458,297.599283,815.896647


In [88]:
len(a)

18

In [142]:
Xtr

array([[69.0, 206.0, 142.0, ..., 335, 89, 89],
       [3.0, 15.0, 10.0, ..., 149, 103, 18],
       [203.0, 611.0, 407.0, ..., 235273, 35954, 0],
       ..., 
       [153.0, 342.0, 209.0, ..., 685384, 14574, 31],
       [0.0, 6.0, 4.0, ..., 12, 12, 0],
       [2.0, 29.0, 20.0, ..., 47, 15, 0]], dtype=object)

In [30]:
from sklearn.externals import joblib

In [418]:
joblib.dump(boostmodel1,"0.51_ScoreModel.pkl",compress=9)

['0.51_ScoreModel.pkl']

In [ ]:
joblib.dump(model,"0.443_ScoreModel.pkl",compress=9)

In [71]:
def create_submission(X, name):
    np.savetxt('%s.csv'%name, X, delimiter=',', fmt="%d", header='user_id,passed', comments='')

In [72]:
#Предскажем результат и запишем его в .csv
#ans = clf.best_estimator_.predict(X_test)
ans = boostmodel1.predict(X_test)
result = np.concatenate((np.asarray(ind_test, dtype=int).reshape(-1, 1), np.asarray(ans, dtype=int).reshape(-1, 1)), axis=1)
create_submission(result, 'submission')

In [1]:
events["user_id"].unique()#events["user_id"].unique()#.head()

NameError: name 'events' is not defined

In [9]:

events.head()

,user_id,action,step_id,time,step_type,step_cost,discover,view,start,pass
0,5920,discovered,2681,1403165936,text,0,1.0,1.0,1.0,0.0
1,5920,passed,2681,1403165936,text,0,1.0,0.0,1.0,1.0
2,5920,viewed,2681,1403165936,text,0,1.0,1.0,0.0,0.0
3,5920,passed,2682,1403165946,text,0,1.0,0.0,1.0,1.0
4,5920,viewed,2682,1403165946,text,0,1.0,1.0,0.0,0.0


In [149]:
structure.head()

,course_id,module_id,module_position,lesson_id,lesson_position,step_id,step_position,step_type,step_cost
0,7,17,1,2721,1,7225,1,text,0
1,7,17,1,2721,1,7230,2,text,0
2,7,17,1,2721,1,7231,3,text,0
3,7,17,1,2721,1,7232,4,text,0
4,7,17,1,2721,1,43604,5,text,0


In [151]:
structure["step_type"].unique()

array(['text', 'video', 'choice', 'code', 'number', 'string'], dtype=object)

In [ ]:
#Gather statistics only bout costed
#Bool for code, string, number ...